In [4]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])


In [5]:
import folium
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [9]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [10]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [11]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [13]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, site in launch_sites_df.iterrows():
    circle = folium.Circle([site['Lat'], site['Long']], color='#d35400', radius=50, fill=True).add_child(folium.Popup(site['Launch Site']))
    marker = folium.Marker(
    [site['Lat'], site['Long']],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

In [14]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [15]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    # If class=1, marker_color value will be green
    # If class=0, marker_color value will be red
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [16]:
#put launches into clusters and put onto map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster) #note, this won't show up unless the individual launches are put onto the map (the next lines of code cover this)


for index, row in spacex_df.iterrows():
    
    #define the position and launch site for each launch
    lat = row['Lat']
    long = row['Long']
    launch_site = row['Launch Site']
    
    #deine the launch marker for each launch
    launch_marker = folium.Marker(
        [lat, long],
        # Create an icon as a text label and colour depending on launch success/failure (marker colour)
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    )
    
    # Add the launch marker for each launch to the launch clusters (not the map, the launch clusters themselves)
    marker_cluster.add_child(launch_marker)

#show the map zoomed out with the centre at NASA
site_map

In [17]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [19]:

# find coordinate of the closet coastline
launch_site_lat = 28.56319
launch_site_long = -80.57683
launch_site_coordinates = [launch_site_lat, launch_site_long]

coastline_lat = launch_site_lat #same latitude, so the line is directly east
coastline_long = -80.56794
coastline_coordinates = [coastline_lat, coastline_long]

distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_lat, coastline_long)

In [20]:
print(f"The coastline is {distance_coastline:.2f} km due East from the launch site")

The coastline is 0.87 km due East from the launch site


In [21]:
#define the distance marker (which will be shown at the coastline)
distance_marker = folium.Marker(
    coastline_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#ff5c33;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
#add the distance marker to the map and show the map
site_map.add_child(distance_marker)
site_map

In [22]:
#define the distance line (between the coastline and the launch site)
distance_line=folium.PolyLine(
    locations=[launch_site_coordinates, coastline_coordinates],
    weight=1
)

#add the distance line to the map and show the map
site_map.add_child(distance_line)
site_map

In [23]:
def distance_and_line(destination_coordinates):
    # Create a marker with distance to a closest city, railway, highway, etc.
    # Draw a line between the marker to the launch site
    
    #calculate the distance between the chosen destination and the launch site defined previously, using the calculate_distance function 
    distance_to_destination = calculate_distance(launch_site_lat, launch_site_long, destination_coordinates[0], destination_coordinates[1])
    
    
    #define the distance marker (which will be shown at the chosen destination)
    distance_marker = folium.Marker(
        destination_coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#ff5c33;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_destination),
            )
        )

    #define the distance line (between the chosen destination and the launch site)
    distance_line=folium.PolyLine(
        locations=[launch_site_coordinates, destination_coordinates],
        weight=1
    )

    #add the distance marker to the map 
    site_map.add_child(distance_marker)
    #add the distance line to the map 
    site_map.add_child(distance_line)

    
closest_city_coordinates = [28.10106, -80.6369]
closest_railway_coordinates = [28.57208, -80.58528]
closest_highway_coordinates = [28.56327, -80.57076]

#add distance markers and lines for closest city, railway and highway, using the above function
distance_and_line(closest_city_coordinates)
distance_and_line(closest_railway_coordinates)
distance_and_line(closest_highway_coordinates)

#show the map
site_map